In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'svg'

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
import os
import numpy as np
from torchsummary import summary
from torchvision import transforms                                                                                                                                        
from torchvision import models
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 600
from PIL import Image
import pandas as pd

import sys
sys.path.append("../../extra/pytorch-cnn-visualizations/src")
import deep_dream

sys.path.append("../../src/")
sys.path.append("../")
from datasets import imagenet

from layer_activation_with_guided_backprop import GuidedBackprop
from misc_functions import save_gradient_images
from aux.utils import obtain_features_map, load_imgs, zscore, extract_valid
from aux.visualization import visualize_features_map
from aux.visualization import visualize_features_map_for_comparision
from utils.visualizations.visualize import concat_imgs, preprocess_arrays

## Hyperparameters.

## Get Image Path

In [3]:
def main(exp, cnn_layer, filter_pos, class_index, device):
    num_class = 30
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    reverse_mean = [-0.485, -0.456, -0.406]
    reverse_std = [1/0.229, 1/0.224, 1/0.225]
    train_transform = transforms.Compose([                                                                                                                                
        transforms.Resize((224, 224), interpolation=Image.BILINEAR),                                                                                              
        transforms.ToTensor(),                                                                                                                                            
        #transforms.Normalize(mean, std)                                                                                                                                   
           ])         
    trainset = imagenet.ImageNet(root="/media/lincolnzjx/HardDisk/Datasets/", 
                                 is_train=True, transform=train_transform)
    
    trainset.set_data([class_index], num_class)
    imgs_path = []                                                                                                                                                            
    images = []
    labels = []
    for img, label, img_path in trainset:                                                                                                                                     
        images.append(img.unsqueeze(0))                                                                                                                                       
        labels.append(label)                                                                                                                                                  
        imgs_path.append(img_path)  
        
        
    save_dir = "./generated/"
    ################### Hyper-Parameter #######################
    # exp = resume_exp
    # epoch = resume_epoch
    ##########################################################
    ab_path = os.path.join(save_dir, exp)
    
    index2image = {index: item.split("/")[-1].split(".")[0] for index, item in enumerate(imgs_path)}
    index2image
    
    images_cpu = np.array([image.detach().clone().cpu().numpy().squeeze() for image in images])
    
    # Load image
    # ext may be different.
    #optimized_data, valid_imgs_path, valid_imgs_index = load_imgs(ab_path, imgs_path, non_exists_ok=True, ext=".png")
    #valid_imgs, valid_labels = extract_valid(images, labels, valid_imgs_index)
    #optimized_data_zscore = zscore(optimized_data, mean, std)
    images_zscore = zscore(images_cpu, mean, std)
    
    # Move to device
    # opt_image = torch.from_numpy(optimized_data_zscore).to(device)
    original_image = torch.from_numpy(images_zscore).to(device)
    prep_imgs = original_image
    ## Create some need path
    ################### Hyper-Parameter #######################
    # exp = resume_exp
    # epoch = resume_epoch
    ##########################################################
    index2image = {index: item.split("/")[-1].split(".")[0] for index, item 
                   in enumerate(imgs_path)}
    
    
    pretrained_model = models.vgg16(pretrained=True).to(device)
    GBP = GuidedBackprop(pretrained_model) 
    
    dir_path = os.path.join(save_dir, exp)
    if not os.path.exists(dir_path):
        os.mkdir(dir_path)
        
    for prep_img, img_path in zip(prep_imgs, imgs_path):
        prep_img = prep_img.unsqueeze(dim=0)
        prep_img.requires_grad_(True)
        guided_grads = GBP.generate_gradients(prep_img, None, cnn_layer, 
                                              filter_pos)
        
        name = img_path.split("/")[-1].split(".")[0] + ".png"
        file_name_to_export = os.path.join(dir_path, "GuidedBPcolor_" + name)
        save_gradient_images(guided_grads, file_name_to_export)

## Main

### 14

In [4]:
excel_path = "../analysis/week9/Batches.xlsx"
sheet = "Others"
excepts = ["052148"]

In [5]:
xlsx = pd.ExcelFile(excel_path)
excel = pd.read_excel(xlsx, sheet, Sdtype={"exp": str, 
                                           "selected_layer": int,
                                           "selected_filter": int, 
                                           "class_index": int,
                                           "Done": str}).values

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

for data in excel:
    exp, selected_layer, selected_filter, class_index, done = data[0], data[1], \
        data[2], data[3], data[4]
    exp = exp.replace("x", "")
    if exp in excepts:
        print("=> Skip")
    else:
        if done == "N":
            print("=> RUN ", exp, selected_layer, selected_filter, 
                  class_index)
            main(exp, selected_layer, selected_filter, class_index, device)
            print("")
            print("-"*50)

=> RUN  052145 6 19 968
Len of new dataset is :30
(30, 3, 224, 224)

--------------------------------------------------
=> RUN  052146 6 19 878
Len of new dataset is :30
(30, 3, 224, 224)

--------------------------------------------------
=> RUN  052147 6 19 90
Len of new dataset is :30
(30, 3, 224, 224)

--------------------------------------------------
=> Skip
=> RUN  052195 3 20 968
Len of new dataset is :30
(30, 3, 224, 224)

--------------------------------------------------
=> RUN  052196 3 20 878
Len of new dataset is :30
(30, 3, 224, 224)

--------------------------------------------------
=> RUN  052197 3 20 90
Len of new dataset is :30
(30, 3, 224, 224)

--------------------------------------------------
=> RUN  052198 3 20 14
Len of new dataset is :30
(30, 3, 224, 224)

--------------------------------------------------
=> RUN  052201 1 16 968
Len of new dataset is :30
(30, 3, 224, 224)

--------------------------------------------------
=> RUN  052202 1 16 90
Len of new d

../../extra/pytorch-cnn-visualizations/src/misc_functions.py:47: RuntimeWarning:

invalid value encountered in true_divide



TypeError: Cannot handle this data type: (1, 1, 3), <f4

### 950

In [6]:
excel_path = "../analysis/week9/Batches.xlsx"
sheet = "950"
excepts = ["052148"]

In [7]:
xlsx = pd.ExcelFile(excel_path)
excel = pd.read_excel(xlsx, sheet, Sdtype={"exp": str, 
                                           "selected_layer": int,
                                           "selected_filter": int, 
                                           "class_index": int,
                                           "Done": str}).values

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

for data in excel:
    exp, selected_layer, selected_filter, class_index, done = data[0], data[1], \
        data[2], data[3], data[4]
    exp = exp.replace("x", "")
    if exp in excepts:
        print("=> Skip")
    else:
        if done == "N":
            print("=> RUN ", exp, selected_layer, selected_filter, 
                  class_index)
            main(exp, selected_layer, selected_filter, class_index, device)
            print("")
            print("-"*50)

=> RUN  061300 1 16 950
Len of new dataset is :30
(30, 3, 224, 224)

--------------------------------------------------
=> RUN  052490 1 16 950
Len of new dataset is :30
(30, 3, 224, 224)

--------------------------------------------------
=> RUN  7001 1 47 950
Len of new dataset is :30
(30, 3, 224, 224)

--------------------------------------------------
=> RUN  7004 3 28 950
Len of new dataset is :30
(30, 3, 224, 224)

--------------------------------------------------
=> RUN  060490 3 20 950
Len of new dataset is :30
(30, 3, 224, 224)

--------------------------------------------------
=> RUN  7005 6 19 950
Len of new dataset is :30
(30, 3, 224, 224)

--------------------------------------------------
=> RUN  7006 8 99 950
Len of new dataset is :30
(30, 3, 224, 224)

--------------------------------------------------
=> RUN  70071 11 75 950
Len of new dataset is :30
(30, 3, 224, 224)

--------------------------------------------------
=> RUN  70081 13 91 950
Len of new dataset is :3